# Location Designator
This example will show you waht location designator are, how to use them and what they are capable of. 

Location Deisgnator are used to semantically describe locations in the world. You could, for example, create a location designator that describes every position where a robot can be placed without colliding with the environment. Location designator can describe locations for:
   
   * Occupany 
   * Reachability   
   * Visibility 
   * ...and more

To find locations that fit the given constrains, location designator create Costmaps. Costmaps are a 2D distribution that have a value greater than 0 for every position that fits the costmap criteria.

Location designator work similar to other designator, meaning you have to create a location designator description which describes the location. This description can then be resolved to the actual 6D pose on runtime.

In [ ]:
from pycram.bullet_world import BulletWorld
from pycram.pose import Pose

world = BulletWorld()

The following visualizes the robot, environment and objects to RvizWeb.
(Skip if running locally.)

In [ ]:
from pycram.ros.tf_broadcaster import TFBroadcaster
from pycram.ros.viz_marker_publisher import VizMarkerPublisher

tf_broadcaster = TFBroadcaster()
viz_publisher = VizMarkerPublisher()

## Occupancy

We will start with a simple location designator that describes a location where the robot can stand without colliding with the environment.

In [ ]:
from pycram.bullet_world import Object

kitchen = Object("kitchen", "environment", "kitchen_no_walls.urdf")

In [ ]:
from pycram.designators.motion_designator import MoveMotion
from pycram.process_module import simulated_robot

pr2 = Object("pr2", "robot", "pr2.urdf")
pr2.set_joint_state("torso_lift_joint", 0.2)

Next up we will create the location designator description, the ```CostmapLocation``` that we will be using needs a target as parameter. This target describes what the location designator is for, this could either be a pose or object that the robot should be able to see or reach.

In this case we only want poses where the robot can be placed, this is the default behaviour of the location designator which we will be extending later. 

In [ ]:
from pycram.designators.location_designator import CostmapLocation

location_description = CostmapLocation(Pose([0, 0, 0], [0, 0, 0, 1]))

pose = location_description.resolve()

print(pose)

Or define the designator resolution iteratively.

In [ ]:
location_description = iter(CostmapLocation(Pose()))

In [ ]:
print(next(location_description).pose)

## Reachable
Next we want to locations from where the robot can reach a specific point, like an object the robot should pick up. This can also be done with the ```CostmapLocation``` description, but this time we need to provide an additional argument. The additional argument is the robo which should be able to reach the pose. 

Since a robot is needed we will use the PR2 and use a milk as a target point for the robot to reach. The torso of the PR2 will be set to 0.2 since otherwise the arms of the robot will be too low to reach on the countertop.


In [ ]:
milk = Object("milk", "milk", "milk.stl", pose=Pose([1.3, 1, 0.9]))

In [ ]:
from pycram.designators.object_designator import BelieveObject

pr2.set_joint_state("torso_lift_joint", 0.2)

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()
location_description = CostmapLocation(target=target, reachable_for=robot_desig)

print(location_description.resolve())

As you can see we get a pose near the countertop where the robot can be placed without colliding with it. Furthermore, we get a list of arms with which the robot can reach the given object.

## Visible
The ```CostmapLocation``` can also find position from which the robot can see a given object or location. This is very similar to how rechable locations are described, meaning we provide a object designator or a pose and a robot designator but this time we use the ```visible_for``` parameter.  

For this example we need the milk as well as the PR2, so if you did not spawn them during the previous location designator you can spawn them with the following cell.  

In [ ]:
location_description = CostmapLocation(target=target, visible_for=robot_desig)

print(location_description.resolve())

In [ ]:
location_description = iter(CostmapLocation(target=target, visible_for=robot_desig))

print(next(location_description))

In [ ]:
print(next(location_description))

In [ ]:
target = next(location_description)

In [ ]:
target.pose

## Semantic 
Semantic location designator are used to create location descriptions for semantic entities, like a table. An example of this is: You have a robot that picked up an object and should place it on a table. Semantic location designator then allows to find poses that are on this table.

Semantic location designator need an object from which the target entity is a part and the urdf link representing the entity. In this case we want a position on the kitchen island, so we have to provide the kitchen object designator since the island is a part of the kitchen and the link name of the island surface. 

For this example we need the kitchen as well as the milk. If you spawned them in one of the previous examples you don't need to execute the following cell.

In [ ]:
from pycram.designators.location_designator import SemanticCostmapLocation 
from pycram.designators.object_designator import BelieveObject

kitchen_desig = BelieveObject(names=["kitchen"]).resolve()
milk_desig = BelieveObject(names=["milk"]).resolve()

location_description = SemanticCostmapLocation(urdf_link_name="kitchen_island_surface", 
                                               part_of=kitchen_desig,
                                              for_object=milk_desig)

print(location_description.resolve())

## Location Designator as Generator
Location designator descriptions implement an iter method, so they can be used as generators which generate valid poses for the location described in the description. This can be useful if the first pose does not work for some reason. 

In [ ]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, visible_for=robot_desig)

for pose in location_description:
    print(pose.pose)

## Accessing Locations
Accessing describes a location from which the robot can open a drawer. The drawer is specified by a ObjectPart designator which describes the handle of the drawer.

At the moment this location designator only works in the apartment environment, so please remove the kitchen if you spawned it in a previous example. Furthermore, we need a robot so we also spawn the pr2 if it isn't spawned already.

In [ ]:
kitchen.remove()

In [ ]:
apartment = Object("apartment", "environment", "apartment.urdf")

In [ ]:
pr2 = Object("pr2", "robot", "pr2.urdf")
pr2.set_joint_state("torso_lift_joint", 0.25)

In [ ]:
from pycram.designators.object_designator import *
from pycram.designators.location_designator import *

apartment_desig = BelieveObject(names=["apartment"])
handle_desig = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig.resolve())
robot_desig = BelieveObject(names=["pr2"])

access_location = AccessingLocation(handle_desig.resolve(), robot_desig.resolve()).resolve()
print(access_location.pose)